Gather fit results for feature reinforcement learning model into a dataframe. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import pickle 
from scipy import stats
from scipy.special import logsumexp

# Custom dependencies
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()) + '/libraries/')

## 1. Select fit type

In [2]:
# Options: 'choice', 'gaze-no-cb', 'gaze-cb', 'joint-no-cb', 'joint-cb'
fit_type = 'gaze-no-cb'

## 2. Gather fit results in a dataframe

In [3]:
## Set subjects. 
# subjects = np.arange(24)+1
# subjects = np.arange(1)+1
subjects = [8]
excluded_subjects = [5,9,18]
# subjects = subjects[np.isin(subjects, excluded_subjects, invert=True)]
n_subj = len(subjects)

## Set model fits path.
if fit_type == 'choice':
    fit_paths = '/Users/angelaradulescu/Dropbox/research/thesis/code/analysis/results/2.MLE/FRL_choice/20200917/'
if fit_type == 'gaze-no-cb': 
    fit_paths = '/Users/angelaradulescu/Dropbox/research/thesis/code/analysis/results/2.MLE/FRL_att_no_center_bias/20200914/'
if fit_type == 'gaze-cb': 
    fit_paths = '/Users/angelaradulescu/Dropbox/research/thesis/code/analysis/results/2.MLE/FRL_att_center_bias_al/20210810/'
if fit_type == 'joint-no-cb': 
    fit_paths = '/Users/angelaradulescu/Dropbox/research/thesis/code/analysis/results/2.MLE/FRL_joint/20200412/'
if fit_type == 'joint-cb': 
    fit_paths = '/Users/angelarradulescu/Dropbox/research/thesis/code/analysis/results/2.MLE/FRL_joint_center_bias/20200909/'
    
## Gather fit results in a dataframe. 
fit_all = []
for s in np.arange(n_subj):

    fit_subj = []
    
    ## Load fit for this subject.
    subj_id = subjects[s]
    print('subject: ' + str(subj_id))
    fit_path = fit_paths + 'subj' + str(subj_id) + '_MLE_fit.obj'
    fit_file = open(fit_path,'rb')
    fit = pickle.load(fit_file)
    
    n_iter, a, b = fit.results.shape
    n_parms = len(fit.initial_conditions[0,:])
    
    ## Loop through iterations.
    for i in np.arange(n_iter):
        
        fit_iter = []
        
        ## Append subject and iteration number.
        fit_iter.append([subj_id, i+1])
        
        ## Print initial conditions for this iteration. 
        init_conds = fit.initial_conditions[i,:]
        fit_iter.append(init_conds)
        
        ## Average fit parameter values in this iteration.
        fit_params = np.mean(fit.results[i,:,0:n_parms],axis=0)
        fit_iter.append(fit_params)
        
        ## Average likelihood from all games in this iteration. 
        cv_log_lik = np.mean(fit.results[i,:,-1])
        fit_iter.append(cv_log_lik)

        fit_subj.append(np.hstack(fit_iter))
        
    fit_all.append(np.vstack(fit_subj))
     
model_fit_results = pd.DataFrame(np.vstack(fit_all))

if fit_type == 'choice':
    model_fit_results.columns = ['subj',
                                 'iter',
                                 'learning_rate_init', 
                                 'decay_rate_init', 
                                 'beta_value_choice_init', 
                                 'learning_rate', 
                                 'decay_rate', 
                                 'beta_value_choice', 
                                 'log_lik']

if fit_type == 'gaze-no-cb':
    model_fit_results.columns = ['subj',
                                 'iter',
                                 'learning_rate_init', 
                                 'decay_rate_init', 
                                 'beta_value_gaze_init', 
                                 'precision_init', 
                                 'learning_rate', 
                                 'decay_rate', 
                                 'beta_value_gaze', 
                                 'precision', 
                                 'log_lik']

if fit_type == 'gaze-cb':
    model_fit_results.columns = ['subj',
                                 'iter',
                                 'learning_rate_init', 
                                 'decay_rate_init', 
                                 'beta_value_gaze_init',
                                 'beta_center_dim_init',
                                 'beta_center_feat_init',
                                 'precision_init', 
                                 'learning_rate', 
                                 'decay_rate', 
                                 'beta_value_gaze',
                                 'beta_center_dim',
                                 'beta_center_feat',
                                 'precision', 
                                 'log_lik']

if fit_type == 'joint-no-cb':
    model_fit_results.columns = ['subj','iter',
                                 'learning_rate_init', 
                                 'decay_rate_init', 
                                 'beta_value_choice_init', 
                                 'beta_value_gaze_init', 
                                 'precision_init', 
                                 'learning_rate', 
                                 'decay_rate', 
                                 'beta_value_choice', 
                                 'beta_value_gaze', 
                                 'precision', 
                                 'log_lik']

if fit_type == 'joint-cb':
    model_fit_results.columns = ['subj','iter',
                                 'learning_rate_init', 
                                 'decay_rate_init', 
                                 'beta_value_choice_init', 
                                 'beta_value_gaze_init',
                                 'beta_center_dim_init',
                                 'beta_center_feat_init',
                                 'precision_init', 
                                 'learning_rate', 
                                 'decay_rate', 
                                 'beta_value_choice', 
                                 'beta_value_gaze', 
                                 'beta_center_dim',
                                 'beta_center_feat',
                                 'precision', 
                                 'log_lik']    

# model_fit_results.to_csv(fit_paths + 'results.csv')

subject: 8


In [4]:
bf = model_fit_results.loc[model_fit_results.groupby('subj')['log_lik'].transform(max) == model_fit_results['log_lik']].reset_index(drop=True)

In [5]:
bf

,subj,iter,learning_rate_init,decay_rate_init,beta_value_gaze_init,precision_init,learning_rate,decay_rate,beta_value_gaze,precision,log_lik
0,8.0,1.0,0.982119,0.914708,3.538857,18.871073,0.128076,0.305595,2.711308,2.882484,359.127257


In [6]:
np.random.randint(3)+1

2

In [14]:
model_fit_results

,subj,iter,learning_rate_init,decay_rate_init,beta_value_gaze_init,precision_init,learning_rate,decay_rate,beta_value_gaze,precision,log_lik
0,8.0,1.0,0.982119,0.914708,3.538857,18.871073,0.128076,0.305595,2.711308,2.882484,359.127257
1,8.0,2.0,0.795160,0.570826,1.776006,9.561228,0.128077,0.305600,2.711287,2.882473,359.127198
2,8.0,3.0,0.019163,0.239360,3.904543,9.246498,0.009400,0.271375,21.247818,2.854265,358.690117
3,8.0,4.0,0.437657,0.398651,4.051959,1.714361,0.128077,0.305598,2.711299,2.882469,359.127219
4,8.0,5.0,0.098534,0.547826,4.369874,10.948715,0.005181,0.278527,35.718170,2.857399,358.718977
